In [ ]:
import numpy as np
import time
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import collections 

### Test graph

In [ ]:
data = pd.read_csv('wiki-topcats-reduced.txt', sep="	", header=None)
data.columns = ["from", "to"]
data.head()

In [ ]:
list1 = data["from"].values

In [ ]:
list2 = data["to"].values

In [ ]:
set1=set(list1)
set2=set(list2)
fin_set = set1.union(set2)

In [ ]:
#number of nodes
len(fin_set)

In [ ]:
#number of edges
len(data)

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_nodes_from(fin_set, categories = [])

In [ ]:
for i in range(len(data)):
    G.add_edge(list1[i], list2[i])

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
categoriesdata = {}
with open('wiki-topcats-categories.txt') as f:
    for i, line in enumerate(f):
        tmp = line.split(';')
        categoriesdata[i] = (tmp[0].replace('Category:', ''), [int(i) for i in tmp[1].split()])
f.close()
categoriesdata

In [ ]:
for elem in categoriesdata:
    for node in categoriesdata[elem][1]:
        if(node in fin_set):
            if len(categoriesdata[elem][1]) > 3500:
                ncat = G.node[node]['categories'] + [categoriesdata[elem][0]]
                G.node[node].update(categories = ncat)

In [ ]:
print(nx.info(G))

In [ ]:
categories = dict()
for tpl in categoriesdata:
    if len(categoriesdata[tpl][1]) > 3500:
        categories[categoriesdata[tpl][0]] = [node for node in categoriesdata[tpl][1] if node in fin_set]

In [ ]:
input_category = "Year_of_birth_unknown"

In [ ]:
from multiprocessing import Pool

In [ ]:
inf=float("inf")

for category in categories:
    file=open("/home/jagg/Data/HW5/files/" + category, "w")
    file.close()

print("Sources:\n")

def compute(source):

    print(source)

    level=0

    current_level=set()
    next_level=set()
    current_level.add(source)
    visited=set()
    
    for link in list(G.neighbors(source)):
        next_level.add(link)
    
    while next_level!=set():
    
        next_level=set()
    
        for node in current_level:
            visited.add(node)

            if node == source:
                for category in G.node[node]['categories']:
                    file=open("/home/jagg/Data/HW5/files/" + category, "a")
                    file.write("0")
                    file.write("\n")
                    file.close()
                    
            elif node!=source:
                for category in G.node[node]['categories']:
                    file=open("/home/jagg/Data/HW5/files/" + category, "a")
                    file.write(str(level))
                    file.write("\n")
                    file.close()

            for link in list(G.neighbors(node)):
                if link not in visited:
                    next_level.add(link)

        level+=1
        current_level=next_level

pool = Pool()
pool.map(compute, categories[input_category])
pool.close()
pool.join()

In [ ]:
category_dict=dict()

In [ ]:
for category in categories:
    
    file=open("/home/jagg/Data/HW5/files/" + category, "r")
    content=file.read()
    content=content.split("\n")
    content.remove("")
    file.close()
    content=list(map(float, content))
    
    missing_infinities=len(categories[input_category])*len(categories[category])-len(content)
    
    for infinity in range(missing_infinities):
        content.append(inf)
        
    median=np.median(content)
    category_dict[category]=median
    
print(category_dict)

In [ ]:
my_rank=collections.namedtuple('Category', ["name", "score"])

In [ ]:
category_list=list()

for category in category_dict:
    category_list.append(my_rank(name=category, score=category_dict[category]))

print(category_list)

In [ ]:
for i in range(len(category_list)-1):
    for j in range(len(category_list)-1):
        if category_list[j].score > category_list[j+1].score:
            temp=category_list[j]
            category_list[j] = category_list[j+1]
            category_list[j+1]=temp
            
print(category_list)